In [2]:
import cv2
from ipywidgets import Video
import ipywidgets as widgets
from IPython.display import display
import numpy as np

In [3]:
video_path = 'Chess_Data\\chess_hard1.mp4'

In [4]:
video_capture = cv2.VideoCapture(video_path)

In [5]:
def save_first_frame(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read the first frame.")
        return

    # Release the video capture object
    cap.release()

    return frame

first_frame = save_first_frame(video_path)

In [6]:
def find_closest_points_to_corners(points, corners):
    closest_points = []
    for corner in corners:
        distances = np.linalg.norm(points - corner, axis=1)
        closest_point_index = np.argmin(distances)
        closest_points.append(points[closest_point_index])
    return np.array(closest_points)

In [7]:
gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(gray, 50, 150)

frame_with_boxes = first_frame.copy()
dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
dst = cv2.dilate(dst, None)
corners = np.argwhere(dst > 0.01 * dst.max())

# Find corners closest to image corners
image_corners = np.array([[0, 0], [0, first_frame.shape[1]], [first_frame.shape[0], 0], [first_frame.shape[0], first_frame.shape[1]]])
closest_corners = find_closest_points_to_corners(corners, image_corners)

target_square_size = 1000  # Adjust the size as needed
target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)
closest_corners = np.array([[[y, x]] for x, y in closest_corners], dtype=int)

## Tracking

In [8]:
OPENCV_OBJECT_TRACKERS = {
    "boosting": cv2.legacy.TrackerBoosting_create,
    "tld": cv2.legacy.TrackerTLD_create,
    "medianflow": cv2.legacy.TrackerMedianFlow_create,
    "mosse": cv2.legacy.TrackerMOSSE_create
}
# grab the appropriate object tracker using our dictionary of
# OpenCV object tracker objects
# tracker = OPENCV_OBJECT_TRACKERS["csrt"]()

In [9]:
def create_mosse_tracker(image, point):
    tracker = cv2.legacy.TrackerMOSSE_create()
    bbox = (point[0]-40, point[1]-40, 80, 80)
    tracker.init(image, bbox)
    return tracker, bbox

def euclidean_distance(point1, point2):
    x1, y1 = point1[:-2]
    x2, y2 = point2[:-2]
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

def get_relative_bbox(relative_corners, current_corners, failures, id):
    searched_corner = relative_corners[id]

    for i, corner in enumerate(relative_corners):
        if failures[i] or i == id:
            continue

        dx = searched_corner[0] - corner[0]
        dy = searched_corner[1] - corner[1]

        return (current_corners[i][0]+dx, current_corners[i][1]+dy, 80, 80)


In [15]:
video_capture = cv2.VideoCapture(video_path)

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2

ret, frame = video_capture.read()

if not ret:
    print("Error reading video file")
    exit()

trackers = []
current_corners = []
for corner in closest_corners:
    corner = corner[0]
    tracker, bbox = create_mosse_tracker(frame, corner)
    current_corners.append(bbox)
    trackers.append(tracker)
    
previous_corners = current_corners.copy()
while True:
    failures = [False, False, False, False]
    # Read a new frame
    ret, frame = video_capture.read()

    # Break the loop if the video has ended
    if not ret:
        break

    # Update the MOSSE tracker
    dist_thresh = 20
    for i, tracker in enumerate(trackers):
        success, bbox = tracker.update(frame)
    
        # Draw bounding box around the tracked object
        if success:
            bbox = tuple(map(int, bbox))
            current_corners[i] = bbox

            if euclidean_distance(previous_corners[i], current_corners[i]) > dist_thresh:
                failures[i] = True
        else:
            failures[i] = True
                        
    for i in range(4):
        if failures[i]:
            bbox = get_relative_bbox(previous_corners, current_corners, failures, i)
            previous_corners[i] = bbox
            trackers[i].init(frame, bbox)
        else:
            bbox = current_corners[i]
            previous_corners[i] = bbox

        center_x = bbox[0] + bbox[2] // 2
        center_y = bbox[1] + bbox[3] // 2

        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 4)
        cv2.putText(frame, f"T{i}", (center_x-10, center_y), font, font_scale, (0, 255, 0), thickness)

    # Display the frame
    cv2.imshow('MOSSE Tracker', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
video_capture.release()
cv2.destroyAllWindows()


intializing 1
intializing 1
fail 1
intializing 1
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intializing 0
fail 0
intial

In [19]:
video_capture = cv2.VideoCapture(video_path)

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2

ret, frame = video_capture.read()

if not ret:
    print("Error reading video file")
    exit()

trackers = []
# initial_corners = []
last_good_bboxes = []
for corner in closest_corners:
    corner = corner[0]
    # initial_corners.append(corner)
    tracker, bbox = create_mosse_tracker(frame, corner)
    trackers.append(tracker)
    last_good_bboxes.append(bbox)
    
frame_hist = []
hist_len = 120
while True:
    # Read a new frame
    ret, frame = video_capture.read()
    frame_hist.append(frame)
    if len(frame_hist) > hist_len:
        frame_hist.pop(0)

    # Break the loop if the video has ended
    if not ret:
        break

    # new_corners = []

    # Update the MOSSE tracker
    for i, tracker in enumerate(trackers):
        success, bbox = tracker.update(frame)
    
        # Draw bounding box around the tracked object
        if success:
            bbox = tuple(map(int, bbox))
            last_good_bboxes[i] = bbox
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 4)
            
            center_x = bbox[0] + bbox[2] // 2
            center_y = bbox[1] + bbox[3] // 2
            
            cv2.putText(frame, f"T{i}", (center_x-10, center_y), font, font_scale, (0, 255, 0), thickness)

            # new_corners.append([center_x, center_y])

        else:
            print(f"Fail tracker {i}")
            for f in frame_hist:
                success, bbox = tracker.update(f)
                if success:
                    bbox = tuple(map(int, bbox))
                    cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 4)
                    break


    # initial_corners = np.array(new_corners)

    # Display the frame
    cv2.imshow('MOSSE Tracker', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
video_capture.release()
cv2.destroyAllWindows()


Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 1
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail tracker 0
Fail track

## LK tracking

In [15]:
# # Load the video
# cap = cv2.VideoCapture(video_path)

# # Parameters for Lucas-Kanade optical flow
# lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# # Read the first frame
# ret, first_frame = cap.read()
# gray_first_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Calculate optical flow using Lucas-Kanade method
#     new_corners, status, _ = cv2.calcOpticalFlowPyrLK(
#         gray_first_frame, gray_frame, np.float32(closest_corners), None, **lk_params)

#     # Filter valid points
#     closest_corners = closest_corners.astype(int)
#     good_new = new_corners[status == 1]
#     good_old = closest_corners[status == 1]

#     # Draw the corners on the frame
#     for i, (new, old) in enumerate(zip(good_new, good_old)):
#         a, b = old.ravel()
#         c, d = new.ravel()
#         frame = cv2.circle(frame, (int(c), int(d)), 20, (0, 255, 0), -1)

#     # Display the frame with the corners
#     cv2.imshow('Corners Detected', frame)

#     # Break the loop if 'q' key is pressed
#     if cv2.waitKey(25) & 0xFF == ord('q'):
#         break


# # Release resources
# cap.release()
# cv2.destroyAllWindows()


## SIFT matching test

In [ ]:
# video_capture = cv2.VideoCapture(video_path)
# ret, frame = video_capture.read()
# if not ret:
#     print("Error")

# homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)

# # Apply the perspective transformation
# calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))
# calibrated_image = cv2.cvtColor(calibrated_image, cv2.COLOR_BGR2GRAY)

# sift = cv2.SIFT_create()

# FLANN_INDEX_KDTREE = 1
# index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
# search_params = dict(checks = 50)
# flann = cv2.FlannBasedMatcher(index_params, search_params)

# MIN_MATCH_COUNT = 10

# while True:
#     ret, frame = video_capture.read()
#     if not ret:
#         break

#     frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
#     kp1, des1 = sift.detectAndCompute(calibrated_image, None)
#     kp2, des2 = sift.detectAndCompute(frame_gray, None)
    
#     matches = flann.knnMatch(des1, des2, k=2)
#     good = []
#     for m,n in matches:
#         if m.distance < 0.7*n.distance:
#             good.append(m)

#     if len(good) > MIN_MATCH_COUNT:
#         blurred = cv2.GaussianBlur(frame_gray, (5, 5), 0)
#         edges = cv2.Canny(frame_gray, 50, 150)

#         frame_with_boxes = frame.copy()
#         dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
#         dst = cv2.dilate(dst, None)
#         corners = np.argwhere(dst > 0.01 * dst.max())

#         # Find corners closest to image corners
#         image_corners = np.array([[0, 0], [0, frame.shape[1]], [frame.shape[0], 0], [frame.shape[0], frame.shape[1]]])
#         closest_corners = find_closest_points_to_corners(corners, image_corners)

#         target_square_size = 1000  # Adjust the size as needed
#         target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)
#         closest_corners = np.array([[[y, x]] for x, y in closest_corners], dtype=int)
#         homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)

#         # Apply the perspective transformation
#         calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))

#     else:
#         print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
#         matchesMask = None
    
#     # grid_img = calibrated_image.copy()
#     # offset = 59 # Board edge offset
#     # for x in np.linspace(offset, target_square_size-offset, 9, dtype=int):
#     #     cv2.line(grid_img, (x, 0+offset), (x, target_square_size-offset), color=(0, 0, 255), thickness=2)  
#     #     cv2.line(grid_img, (0+offset, x), (target_square_size-offset, x), color=(0, 0, 255), thickness=2)  

#     # Display the result
#     # cv2.imshow('Chessboard focus', grid_img)
#     cv2.imshow('Chessboard focus', calibrated_image)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# video_capture.release()
# cv2.destroyAllWindows()

Not enough matches are found - 0/10
Not enough matches are found - 0/10
Not enough matches are found - 4/10
Not enough matches are found - 1/10
Not enough matches are found - 1/10
Not enough matches are found - 3/10
Not enough matches are found - 0/10
Not enough matches are found - 1/10
Not enough matches are found - 1/10
Not enough matches are found - 1/10
Not enough matches are found - 1/10
Not enough matches are found - 1/10
Not enough matches are found - 1/10
Not enough matches are found - 3/10
Not enough matches are found - 1/10


In [ ]:
# video_capture = cv2.VideoCapture(video_path)

# while True:
#     ret, frame = video_capture.read()
#     if not ret:
#         break
    
#     homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)

#     # Apply the perspective transformation
#     calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))

#     grid_img = calibrated_image.copy()
#     offset = 59 # Board edge offset
#     for x in np.linspace(offset, target_square_size-offset, 9, dtype=int):
#         cv2.line(grid_img, (x, 0+offset), (x, target_square_size-offset), color=(0, 0, 255), thickness=2)  
#         cv2.line(grid_img, (0+offset, x), (target_square_size-offset, x), color=(0, 0, 255), thickness=2)  

#     # Display the result
#     cv2.imshow('Chessboard focus', grid_img)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# video_capture.release()
# cv2.destroyAllWindows()